## Event Handling for PySide2 OpenGL Widget

Welcome to the event handling tutorial for the new OpenGL api of PySide2.

By event handling we mean two things: 

- Acquiring the user input through the aid of other widgets and displaying its effect on scene.

- Responding to a state of scene with respect to a condition.

We are going to see an example of the first one in this tutorial. 
For the second one, remember that once a scene is drawn it is as good as gone, because the main use of OpenGL is rendering objects on scene not changing their state.
It is possible to do computation on OpenGL of course and we shall see an example in the next tutorial while dealing with light effects, but it is better to do critical computation at the client code rather than in OpenGL.

This is also evident in the `qt` api as well. 
Simply look at the amount of setters with respect to that of getters, if they exist at all. 
Qt also favors a mindset where you send stuff for rendering only.

Now let's see our final application window, where we finally start to use some of the handles that we had defined from the beginning.

In [2]:
import subprocess

subprocess.run(["python", "app.py"])

CompletedProcess(args=['python', 'app.py'], returncode=0)

It is not very well oriented due to the absence of mouse control but it should give you an idea about how everything works together.

We had also changed the content of the `app.py` to better handle the event mechanism. 

Let's see what's new in `app.py` 

In [ ]:
from PySide2 import QtWidgets
from tutorials.utils.window import GLWindow as AppWindow
from glevents import EventsGL
import sys


class EventAppWindow(AppWindow):
    "Overriding base class with event methods"

    def __init__(self,
                 glwidget: QtWidgets.QOpenGLWidget,
                 parent=None,
                 ):
        super().__init__(glwidget,
                         parent)
        self.camX.setRange(-180.0, 180.0)
        self.camY.setRange(-180.0, 180.0)
        self.xSlider.setRange(-180.0, 180.0)
        self.ySlider.setRange(-180.0, 180.0)
        self.zSlider.setRange(-180.0, 180.0)
        self.upBtn.clicked.connect(self.moveCameraForward)
        self.downBtn.clicked.connect(self.moveCameraBackward)
        self.leftBtn.clicked.connect(self.moveCameraLeft)
        self.rightBtn.clicked.connect(self.moveCameraRight)
        self.camX.valueChanged.connect(self.turnCameraX)
        self.camY.valueChanged.connect(self.turnCameraY)
        self.xSlider.valueChanged.connect(self.rotateCubes)
        self.ySlider.valueChanged.connect(self.rotateCubes)
        self.zSlider.valueChanged.connect(self.rotateCubes)
        #
        self.lastCamXVal = self.camX.value()
        #
        self.lastCamYVal = self.camY.value()

    def moveGLCamera(self, direction: str):
        self.glWidget.moveCamera(direction)

    def moveCameraForward(self):
        self.moveGLCamera("forward")

    def moveCameraBackward(self):
        self.moveGLCamera("backward")

    def moveCameraLeft(self):
        self.moveGLCamera("left")

    def moveCameraRight(self):
        self.moveGLCamera("right")

    def turnCameraX(self, newVal: int):
        "Turn camera around"
        offsetx = newVal - self.lastCamXVal
        valy = self.camY.value() - self.lastCamYVal
        self.glWidget.turnAround(x=float(offsetx),
                                 y=float(valy))
        self.lastCamXVal = newVal

    def turnCameraY(self, newVal: int):
        "Turn camera around"
        offsety = newVal - self.lastCamYVal
        valx = self.camX.value() - self.lastCamXVal
        self.glWidget.turnAround(x=float(valx),
                                 y=float(offsety))
        self.lastCamYVal = newVal

    def rotateCubes(self):
        rx = self.xSlider.value()
        ry = self.ySlider.value()
        rz = self.zSlider.value()
        self.glWidget.rotateCubes(rx, ry, rz)

As you can see, we now have another window which wires the events triggered by other widgets on the window to the glwidget.

GLwidget then simply calls the related method.

Let's see for example what `turnCameraY` method of the window, which is triggered by a value change in `camY` slider, basically the camera slider with `y` label on top, triggers in glwidget.

It calls the `turnAround` method of glwidget with two offset values. Then `turnAround` does the following.

In [ ]:
    def turnAround(self, x: float, y: float):
        ""
        self.camera.lookAround(xoffset=x,
                               yoffset=y,
                               pitchBound=True)
        self.update()


It simply passes these offset values to a camera method. 
You can check `lookAround` method inside `camera.py`, but it simply assigns new `pitch` and `yaw` values using these offsets then updates the vectors of the camera like front, right, up.

More importantly `turnAround` calls the `update` method of the glwidget.
The update method is common for qtwidgets. 
Here it triggers repainting the scene with new values. 
These values happen to modify the orientation of the camera effectively changing the field of visibility.

This has the following implication on code. 
We need to set the data related to triggered events in the `paintGL` rather than `initializeGL`, since `update` simply recalls `paintGL` for repainting the scene, and `initializeGL` is called only once before the first use of `paintGL`.

Let's see now the body of our `paintGL`

In [ ]:
    def paintGL(self):
        "drawing loop"
        funcs = self.context.functions()

        # clean up what was drawn
        funcs.glClear(
            pygl.GL_COLOR_BUFFER_BIT | pygl.GL_DEPTH_BUFFER_BIT
        )
        self.vao.bind()
        self.vbo.bind()

        # actual drawing
        self.program.bind()
        ############ Diff ###############
        # Notice that this exactly the same code
        # that we had used in CubeGL widget
        # I simply copy pasted the same thing
        # to paintGL here.
        # set projection matrix
        projectionMatrix = QMatrix4x4()
        projectionMatrix.perspective(
            self.camera.zoom,
            self.width() / self.height(),
            0.2, 100.0)

        self.program.setUniformValue('projection',
                                     projectionMatrix)

        # set view/camera matrix
        viewMatrix = self.camera.getViewMatrix()
        self.program.setUniformValue('view',
                                     viewMatrix)

        # bind textures
        for i, pos in enumerate(self.cubeCoords):
            #
            cubeModel = QMatrix4x4()
            cubeModel.translate(pos)
            angle = 30 * i
            cubeModel.rotate(angle, self.rotateVector)
            self.program.setUniformValue("model",
                                         cubeModel)
            self.texture1.bind(0)
            self.texture2.bind(1)
            funcs.glDrawArrays(
                pygl.GL_TRIANGLES,
                0,
                36
            )
        self.vbo.release()
        self.program.release()
        self.texture1.release()
        self.texture2.release()

Congragulations, that's it!

Now you know a big part of 3d rendering. 
Most of the stuff from now on would be more or less the same thing with fancier shaders and/or objects and/or transformations.  